In [ ]:
!pip install wfdb
!pip install hrv-analysis
!pip install py-ecg-detectors
!pip install neurokit2

In [ ]:
import pprint
import os
import datetime

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import wfdb
import hrvanalysis
import ecgdetectors
import hrv
import neurokit2 as nk

## Sudden Cardiac Death Holter Database

https://physionet.org/content/sddb/1.0.0/

In [ ]:
# !wget -r -N -c -np https://physionet.org/files/sddb/1.0.0/ -P "/content/drive/MyDrive/Projects/HRV/dataset"

In [ ]:
SDDB_DIR = "/content/drive/MyDrive/Projects/HRV/dataset/physionet.org/files/sddb/1.0.0/"
record = wfdb.rdrecord(os.path.join(SDDB_DIR, "30"))
# record = wfdb.rdrecord("/content/physionet.org/files/sddb/1.0.0/30")
wfdb.plot_wfdb(record=record, title="Sample 30")
pprint.pprint(record.__dict__)

In [ ]:
sddb_clinical_info = {
    "30": {
        "Gender": "M",
        "Age": 43,
        "Underlaying_cardiac_rhythm": "Sinus",
        "Risk": 0
    },
    "31": {
        "Gender": "F",
        "Age": 72,
        "Underlaying_cardiac_rhythm": "Sinus",
        "Risk": 0
    },
    "32": {
        "Gender": "?",
        "Age": 62,
        "Underlaying_cardiac_rhythm": "Sinus with intermittent demand ventricular pacing; CPR at time of cardiac arrest",
        "Risk": 1
    },
    "33": {
        "Gender": "F",
        "Age": 30,
        "Underlaying_cardiac_rhythm": "Sinus",
        "Risk": 0
    },
    "34": {
        "Gender": "M",
        "Age": 34,
        "Underlaying_cardiac_rhythm": "Sinus",
        "Risk": 0
    },
    "35": {
        "Gender": "F",
        "Age": 72,
        "Underlaying_cardiac_rhythm": "Atrial fibrillation",
        "Risk": 1
    },
    "36": {
        "Gender": "M",
        "Age": 75,
        "Underlaying_cardiac_rhythm": "Atrial fibrillation",
        "Risk": 1
    },
    "37": {
        "Gender": "F",
        "Age": 89,
        "Underlaying_cardiac_rhythm": "Atrial fibrillation",
        "Risk": 1
    },
    "38": {
        "Gender": "?",
        "Age": "?",
        "Underlaying_cardiac_rhythm": "Sinus",
        "Risk": 0
    },
    "39": {
        "Gender": "M",
        "Age": 66,
        "Underlaying_cardiac_rhythm": "Sinus",
        "Risk": 0
    },
    "40": {
        "Gender": "M",
        "Age": 79,
        "Underlaying_cardiac_rhythm": "Paced",
        "Risk": 1
    },
    "41": {
        "Gender": "M",
        "Age": "?",
        "Underlaying_cardiac_rhythm": "Sinus",
        "Risk": 0
    },
    "42": {
        "Gender": "M",
        "Age": 17,
        "Underlaying_cardiac_rhythm": "Sinus",
        "Risk": 0
    },
    "43": {
        "Gender": "M",
        "Age": 35,
        "Underlaying_cardiac_rhythm": "Intermittent ventricular pacing",
        "Risk": 1
    },
    "44": {
        "Gender": "M",
        "Age": "?",
        "Underlaying_cardiac_rhythm": "Sinus",
        "Risk": 0
    },
    "45": {
        "Gender": "M",
        "Age": 68,
        "Underlaying_cardiac_rhythm": "Sinus",
        "Risk": 0
    },
    "46": {
        "Gender": "F",
        "Age": "?",
        "Underlaying_cardiac_rhythm": "Sinus",
        "Risk": 0
    },
    "47": {
        "Gender": "M",
        "Age": 34,
        "Underlaying_cardiac_rhythm": "Sinus",
        "Risk": 0
    },
    "48": {
        "Gender": "M",
        "Age": "80",
        "Underlaying_cardiac_rhythm": "Sinus",
        "Risk": 0
    },
    "49": {
        "Gender": "M",
        "Age": 73,
        "Underlaying_cardiac_rhythm": "Sinus with intermittent pacing",
        "Risk": 1
    },
    "50": {
        "Gender": "F",
        "Age": 68,
        "Underlaying_cardiac_rhythm": "Atrial fibrillation",
        "Risk": 1
    },
    "51": {
        "Gender": "F",
        "Age": 67,
        "Underlaying_cardiac_rhythm": "Sinus with intermittent pacing",
        "Risk": 1
    },
    "52": {
        "Gender": "F",
        "Age": 82,
        "Underlaying_cardiac_rhythm": "Sinus",
        "Risk": 0
    },
}

In [ ]:
SAMPLE = ['30', '32', '37', '44', '45', '47', '50']

## Playing Around

In [ ]:
def slice_signal(record_obj, start_sec=0, end_sec=10, fs=250, _print=True):
    start = fs*start_sec
    end = fs*end_sec
    signal = record_obj.p_signal[start:end, 0]

    if _print:
        print("start        :", start)
        print("end          :", end)
        print("len(signal)  :", len(signal))
        print("seconds      :", end_sec - start_sec)

    detectors = ecgdetectors.Detectors(fs)
    # r_peaks = detectors.pan_tompkins_detector(signal)
    # r_peaks = detectors.swt_detector(signal)
    r_peaks = detectors.two_average_detector(signal)
    if _print:
        print("len(r_peaks) :", len(r_peaks))
    return signal, r_peaks

def plot_signal(signal: np.array, r_peaks: list=None, figsize=(6, 4), slice_points=None):
    fig = plt.figure(figsize=figsize)
    plt.plot(signal)
    plt.gca().set_xlabel("")
    plt.gca().set_ylabel("mV")

    # ref: https://stackoverflow.com/questions/24988448/how-to-draw-vertical-lines-on-a-given-plot
    if r_peaks:
        for xc in r_peaks:
            plt.axvline(x=xc, color='red', linestyle='--')

    if slice_points is not None:
        for xc in slice_points:
            plt.axvline(x=xc, color='black', linestyle='-')

    plt.show()

In [ ]:
record_obj = wfdb.rdrecord(os.path.join(SDDB_DIR, "32"))
signal, r_peaks = slice_signal(record_obj, end_sec=5)
plot_signal(signal, r_peaks)
print(r_peaks)

In [ ]:
record_obj = wfdb.rdrecord(os.path.join(SDDB_DIR, "32"))
signal, r_peaks = slice_signal(record_obj, end_sec=10)
plot_signal(signal, r_peaks)
print(r_peaks)

In [ ]:
def rolling_slice(record_obj, start_sec=0, end_sec=20, size_sec=5, fs=250, _plot_signal=True, _print=True):
    full_r_peaks = []
    for _start_sec in range(start_sec, end_sec, size_sec):
        signal, r_peaks = slice_signal(record_obj, start_sec=_start_sec, end_sec=_start_sec+size_sec, fs=fs, _print=_print)
        r_peaks = list(np.array(r_peaks) + _start_sec*fs)
        full_r_peaks += r_peaks
        if _plot_signal:
            plot_signal(signal, r_peaks)
        if _print:
            print(r_peaks)
    if _print:
        print("len(full_r_peaks):", len(full_r_peaks))
    return full_r_peaks

In [ ]:
record = wfdb.rdrecord(os.path.join(SDDB_DIR, "32"))
START_SEC = 0
END_SEC = 60
SIZE_SEC = 10
_PLOT_SIGNAL = False
full_r_peaks_0_60_10 = rolling_slice(record, start_sec=START_SEC, end_sec=END_SEC, size_sec=SIZE_SEC, _plot_signal=_PLOT_SIGNAL)

In [ ]:
record = wfdb.rdrecord(os.path.join(SDDB_DIR, "32"))
START_SEC = 0
END_SEC = 60
SIZE_SEC = 30
_PLOT_SIGNAL = False
full_r_peaks_0_60_30 = rolling_slice(record, start_sec=START_SEC, end_sec=END_SEC, size_sec=SIZE_SEC, _plot_signal=_PLOT_SIGNAL)

In [ ]:
record = wfdb.rdrecord(os.path.join(SDDB_DIR, "32"))
START_SEC = 0
END_SEC = 60
SIZE_SEC = 60
_PLOT_SIGNAL = False
full_r_peaks_0_60_60 = rolling_slice(record, start_sec=START_SEC, end_sec=END_SEC, size_sec=SIZE_SEC, _plot_signal=_PLOT_SIGNAL)

In [ ]:
signal, _ = slice_signal(record_obj=record, start_sec=START_SEC, end_sec=END_SEC, fs=250)
plot_signal(signal=signal, r_peaks=full_r_peaks_0_60_10, figsize=(24, 4))
plot_signal(signal=signal, r_peaks=full_r_peaks_0_60_30, figsize=(24, 4))
plot_signal(signal=signal, r_peaks=full_r_peaks_0_60_60, figsize=(24, 4))

In [ ]:
record = wfdb.rdrecord(os.path.join(SDDB_DIR, "32"))
START_SEC = 0
END_SEC = 10
SIZE_SEC = [2, 5, 10]
FS = 250
_PLOT_SIGNAL = False
_PRINT = False

res = []
slices = []
for _size_sec in SIZE_SEC:
    full_r_peaks_0_10_size = rolling_slice(record, start_sec=START_SEC, end_sec=END_SEC, size_sec=_size_sec, _plot_signal=_PLOT_SIGNAL, _print=_PRINT)
    res.append(full_r_peaks_0_10_size)
    slices.append(np.dot(np.arange(START_SEC, END_SEC+1, _size_sec), FS))

In [ ]:
signal, _ = slice_signal(record_obj=record, start_sec=START_SEC, end_sec=END_SEC, fs=250)
for r_peak, slice_points in zip(res, slices):
    plot_signal(signal=signal, r_peaks=r_peak, figsize=(10, 4), slice_points=slice_points)

In [ ]:
record = wfdb.rdrecord(os.path.join(SDDB_DIR, "32"))
START_SEC = 0
END_SEC = 43200 # 12 jam
SIZE_SEC = 43200//6
FS = 250
_PLOT_SIGNAL = False
_PRINT = False
full_r_peaks = rolling_slice(record, start_sec=START_SEC, end_sec=END_SEC, size_sec=SIZE_SEC, _plot_signal=_PLOT_SIGNAL, _print=_PRINT)
print("="*40, " Summary ", "="*40)
print("seconds      :", END_SEC - START_SEC)
print("splits       :", (END_SEC - START_SEC)//SIZE_SEC)
print("full_r_peaks :", full_r_peaks)
print("len(r_peaks) :", len(full_r_peaks))

In [ ]:
record = wfdb.rdrecord(os.path.join(SDDB_DIR, "32"))
START_SEC = 0
END_SEC = 43200 # 12 jam
SIZE_SEC = 43200//12
FS = 250
_PLOT_SIGNAL = False
_PRINT = False
full_r_peaks = rolling_slice(record, start_sec=START_SEC, end_sec=END_SEC, size_sec=SIZE_SEC, _plot_signal=_PLOT_SIGNAL, _print=_PRINT)
print("="*40, " Summary ", "="*40)
print("seconds      : {0} ({1} hours)".format(END_SEC - START_SEC, (END_SEC - START_SEC)//3600))
print("splits       : {0} ({1} minutes/split)".format((END_SEC - START_SEC)//SIZE_SEC, SIZE_SEC//60))
print("full_r_peaks :", full_r_peaks)
print("len(r_peaks) :", len(full_r_peaks))

In [ ]:
record = wfdb.rdrecord(os.path.join(SDDB_DIR, "32"))
START_SEC = 0
END_SEC = 43200 # 12 jam
SIZE_SEC = 43200//24 # 30 menit
FS = 250
_PLOT_SIGNAL = False
_PRINT = False
full_r_peaks = rolling_slice(record, start_sec=START_SEC, end_sec=END_SEC, size_sec=SIZE_SEC, _plot_signal=_PLOT_SIGNAL, _print=_PRINT)
print("="*40, " Summary ", "="*40)
print("seconds      : {0} ({1} hours)".format(END_SEC - START_SEC, (END_SEC - START_SEC)//3600))
print("splits       : {0} ({1} minutes/split)".format((END_SEC - START_SEC)//SIZE_SEC, SIZE_SEC//60))
print("full_r_peaks :", full_r_peaks)
print("len(r_peaks) :", len(full_r_peaks))

In [ ]:
record = wfdb.rdrecord(os.path.join(SDDB_DIR, "32"))
START_SEC = 0
END_SEC = 43200 # 12 jam
SIZE_SEC = 43200//48 # 15 menit
FS = 250
_PLOT_SIGNAL = False
_PRINT = False
full_r_peaks = rolling_slice(record, start_sec=START_SEC, end_sec=END_SEC, size_sec=SIZE_SEC, _plot_signal=_PLOT_SIGNAL, _print=_PRINT)
print("="*40, " Summary ", "="*40)
print("seconds      : {0} ({1} hours)".format(END_SEC - START_SEC, (END_SEC - START_SEC)//3600))
print("splits       : {0} ({1} minutes/split)".format((END_SEC - START_SEC)//SIZE_SEC, SIZE_SEC//60))
print("full_r_peaks :", full_r_peaks)
print("len(r_peaks) :", len(full_r_peaks))

In [ ]:
record = wfdb.rdrecord(os.path.join(SDDB_DIR, "32"))
START_SEC = 0
END_SEC = 43200 # 12 jam
SIZE_SEC = 43200//144 # 5 menit
FS = 250
_PLOT_SIGNAL = False
_PRINT = False
full_r_peaks = rolling_slice(record, start_sec=START_SEC, end_sec=END_SEC, size_sec=SIZE_SEC, _plot_signal=_PLOT_SIGNAL, _print=_PRINT)
print("="*40, " Summary ", "="*40)
print("seconds      : {0} ({1} hours)".format(END_SEC - START_SEC, (END_SEC - START_SEC)//3600))
print("splits       : {0} ({1} minutes/split)".format((END_SEC - START_SEC)//SIZE_SEC, SIZE_SEC//60))
print("full_r_peaks :", full_r_peaks)
print("len(r_peaks) :", len(full_r_peaks))

In [ ]:
record = wfdb.rdrecord(os.path.join(SDDB_DIR, "30"))
START_SEC = 0
END_SEC = 43200 # 12 jam
SIZE_SEC = 43200//720 # 1 menit
FS = 250
_PLOT_SIGNAL = False
_PRINT = False
full_r_peaks = rolling_slice(record, start_sec=START_SEC, end_sec=END_SEC, size_sec=SIZE_SEC, _plot_signal=_PLOT_SIGNAL, _print=_PRINT)
print("="*40, " Summary ", "="*40)
print("seconds      : {0} ({1} hours)".format(END_SEC - START_SEC, (END_SEC - START_SEC)//3600))
print("splits       : {0} ({1} minutes/split)".format((END_SEC - START_SEC)//SIZE_SEC, SIZE_SEC//60))
print("full_r_peaks :", full_r_peaks)
print("len(r_peaks) :", len(full_r_peaks))

In [ ]:
rri = [full_r_peaks[i+1] - full_r_peaks[i] for i in range(len(full_r_peaks) - 1)]
RMSSD = hrv.HRV(FS).RMSSD(full_r_peaks)
SDNN = hrv.HRV(FS).SDNN(full_r_peaks)
print("RMSSD:", RMSSD)
print("SDNN", SDNN)

In [ ]:
rri = list(np.array(rri) * 1000/250)
nni = hrvanalysis.get_nn_intervals(rri)
print("len(nni)               :", len(nni))
time_domain = hrvanalysis.extract_features.get_time_domain_features(rri)
freq_domain = hrvanalysis.extract_features.get_frequency_domain_features(rri)

In [ ]:
plt.plot(nni)
plt.show()

## Second Attempt

In [ ]:
def slice_signal(record_obj, start_sec=0, end_sec=10, fs=250, _print=True):
    start = fs*start_sec
    end = fs*end_sec
    signal = record_obj.p_signal[start:end, 0]

    if _print:
        print("start        :", start)
        print("end          :", end)
        print("len(signal)  :", len(signal))
        print("seconds      :", end_sec - start_sec)

    detectors = ecgdetectors.Detectors(fs)
    r_peaks = detectors.two_average_detector(signal)
    if _print:
        print("len(r_peaks) :", len(r_peaks))
    return signal, r_peaks

def plot_signal(signal: np.array, r_peaks: list=None, figsize=(6, 4), slice_points=None):
    fig = plt.figure(figsize=figsize)
    plt.plot(signal)
    plt.gca().set_xlabel("")
    plt.gca().set_ylabel("mV")

    # ref: https://stackoverflow.com/questions/24988448/how-to-draw-vertical-lines-on-a-given-plot
    if r_peaks:
        for xc in r_peaks:
            plt.axvline(x=xc, color='red', linestyle='--')

    if slice_points is not None:
        for xc in slice_points:
            plt.axvline(x=xc, color='black', linestyle='-')

    plt.show()

In [ ]:
def rolling_slice(record_obj, start_sec=0, end_sec=20, size_sec=5, fs=250, _plot_signal=True, _print=True):
    full_r_peaks = []
    if end_sec == 'full':
        end_sec = len(record_obj.p_signal[:, 0]) // FS

    for _start_sec in range(start_sec, end_sec, size_sec):
        signal, r_peaks = slice_signal(record_obj, start_sec=_start_sec, end_sec=_start_sec+size_sec, fs=fs, _print=_print)
        r_peaks = list(np.array(r_peaks) + _start_sec*fs)
        full_r_peaks += r_peaks
        if _plot_signal:
            plot_signal(signal, r_peaks)
        if _print:
            print(r_peaks)
    if _print:
        print("len(full_r_peaks):", len(full_r_peaks))
    return full_r_peaks

In [ ]:
start_time = datetime.datetime.now()

sddb = {}

START_SEC = 0
# END_SEC = 3600*18 # 18 jam
END_SEC = "full"
SIZE_SEC = 60 # 1 menit
FS = 250
_PLOT_SIGNAL = False
_PRINT = False
print("FS:", FS)
for record in wfdb.get_record_list("sddb"):
    try:
        print("Record:", record)
        record_obj = wfdb.rdrecord(os.path.join(SDDB_DIR, record))
        print("  signal length          :", record_obj.sig_len)
        print("  recording time (hours) :", record_obj.sig_len / (3600*FS))
        print("  comments               :", record_obj.comments)

        r_peaks = rolling_slice(
            record_obj,
            start_sec=START_SEC,
            end_sec=END_SEC,
            size_sec=SIZE_SEC,
            fs=FS,
            _plot_signal=_PLOT_SIGNAL,
            _print=_PRINT
        )
        # retrieve rr_interval from distance between 2 r_peak points
        rri = [r_peaks[i+1] - r_peaks[i] for i in range(len(r_peaks) - 1)]
        # convert the unit from freq_sample to milisecond
        rri = list(np.array(rri) * 1000 / FS)
        print("  len(r_peaks)           :", len(r_peaks))
        print("  len(rri)               :", len(rri))
        print("  recording time (secs)  :", record_obj.sig_len / FS)
        print("  r_peak / seconds       :", len(r_peaks) / (record_obj.sig_len / FS))

        RMSSD = hrv.HRV(FS).RMSSD(r_peaks)
        SDNN = hrv.HRV(FS).SDNN(r_peaks)
        print("  RMSSD                  :", RMSSD)
        print("  SDNN                   :", SDNN)

        nni = hrvanalysis.get_nn_intervals(rri)
        print("  len(nni)               :", len(nni))
        time_domain = hrvanalysis.extract_features.get_time_domain_features(nni)
        print("  time_domain computed")
        freq_domain = hrvanalysis.extract_features.get_frequency_domain_features(nni)
        print("  freq_domain computed")


        sddb[record] = {
            "id": "sddb-" + str(record),
            "db_source": "sddb",
            "age": sddb_clinical_info[record]["Age"],
            "gender": sddb_clinical_info[record]["Gender"],
            "fs": FS,
            "signal_length": record_obj.sig_len,
            "recording_time_hours": record_obj.sig_len / (3600 * FS),
            "recording_time_seconds": record_obj.sig_len // FS,
            "rri_length": len(rri),
            "nni_length": len(nni),
            **time_domain,
            **freq_domain,
            "cardiac_info": sddb_clinical_info[record]["Underlaying_cardiac_rhythm"],
            "risk": sddb_clinical_info[record]["Risk"],
        }
        print("  RECORD {} SAVED!".format(record))
    except:
        print("\n ERROR (%s)\n" %record)

print("Running time:", datetime.datetime.now() - start_time)

In [ ]:
df_sddb = pd.DataFrame(sddb).T
df_sddb

In [ ]:
df_sddb.to_csv("df_sddb.csv", index=False)
df_sddb.to_excel("df_sddb.xlsx", index=False)

In [ ]:
pd.read_csv("df_sddb.csv")

In [ ]:
pd.read_excel("df_sddb.xlsx")

## Animation Plot

In [ ]:
def animate_plot():
    pass


def rolling_slice(record_obj, start_sec=0, end_sec=20, size_sec=5, fs=250, _plot_signal=True):
    full_r_peaks = []
    for _start_sec in range(start_sec, end_sec, size_sec):
        signal, r_peaks = slice_signal(record_obj, start_sec=_start_sec, end_sec=_start_sec+size_sec, fs=fs)
        r_peaks = list(np.array(r_peaks) + _start_sec*fs)
        full_r_peaks += r_peaks
        if _plot_signal:
            plot_signal(signal, r_peaks)
        print(r_peaks)
    return full_r_peaks